In [291]:
#%logstart -o

import sys

import numpy as np
#in case we need to repeat experiment
np.random.seed(255)

import pandas as pd
pd.options.display.max_rows = 22

import matplotlib.pyplot as plt
plt.style.use('classic')


import seaborn as sns
sns.set()

from sklearn.metrics import classification_report, confusion_matrix 

#Enable
enable_grid_search=False
#Use print instead of display when run as python script
pyscript = True

#Classifier verborsity where supported
verbose_level=1

#Sample Size
sampleN = 1000000

import time




Activating auto-logging. Current session state plus future input saved.
Filename       : ipython_log.py
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : False
State          : active
time.struct_time(tm_year=2018, tm_mon=11, tm_mday=10, tm_hour=17, tm_min=2, tm_sec=49, tm_wday=5, tm_yday=314, tm_isdst=0)
Sat Nov 10 17:02:49 2018


In [ ]:
sys.stdout=open("Initial_Binary_Model.out", "w")

In [ ]:
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ), flush=True)

In [292]:
#df = pd.read_csv('NCDB_2016.csv', engine = 'python')
#df = pd.read_csv('data01_simple.csv', engine = 'python')
df = pd.read_csv('data01_clean.csv', engine = 'python')
#df

In [293]:
print(df.isnull().sum().sum(), flush=True)

0


In [294]:
print(df[df.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum(), flush=True)

0


In [295]:
df_cat = df.astype('category').copy()

In [296]:
df_int = df.astype('int').copy()

In [297]:
print(df.columns, flush=True)

Index(['C_YEAR', 'C_MNTH', 'C_WDAY', 'C_HOUR', 'C_VEHS', 'C_CONF', 'C_RCFG',
       'C_WTHR', 'C_RSUR', 'C_RALN', 'C_TRAF', 'V_ID', 'V_TYPE', 'V_YEAR',
       'P_ID', 'P_SEX', 'P_AGE', 'P_PSN', 'P_SAFE', 'P_USER', 'P_ISEV'],
      dtype='object')


## Convert Class Variable to Binary

In [298]:
## Convert Class Variable to Binary
### Merge Injury and Fatality as a single class
### we will compare the results.
df_binary_class = df_cat.copy()

#perform the conversion in two steps to avoid any unwanted side effects
df_binary_class['P_ISEV'] = df_binary_class['P_ISEV'].map({1: 'safe', 2: 'injury', 3:'fatal'})
df_binary_class['P_ISEV'] = df_binary_class['P_ISEV'].map({'safe': '0', 'injury': '1', 'fatal':'1'})
print((df_binary_class['P_ISEV']=='0').sum(), flush=True)
print((df_binary_class['P_ISEV']=='1').sum(), flush=True)
print(df_binary_class['P_ISEV'].unique(), flush=True)

2568097
3395175
['0' '1']


In [299]:
#df_100k = df_binary_class.sample(n=1000000)
df_100k = df_binary_class.sample(n=sampleN)

## Split Training and Testing for Binary class

In [300]:
#Split between data and class
Y = df_100k[df_binary_class.columns[-1]]
X = df_100k[df_binary_class.columns[0:df_binary_class.columns.size -1]]

#### Split Test(70%) and Train (30%) for Bianry class 

In [301]:
#sprint into train and test 70/30
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [302]:
#print(Xbinary_train, Xbinary_test, Ybinary_train, Ybinary_test)

## Write cleaned data to file for future use.

In [303]:
#lets write the datafile for future use
df_binary_class.to_csv('cleansimple_binary.csv', encoding='utf-8', index=False)

## Clustering based on K-Means Clustering

In [304]:
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ), flush=True)

from sklearn.cluster import KMeans
print("K-Means Clustering: Start", flush=True)
kmeans = KMeans(n_clusters=3, init='random', n_init=10, max_iter=300, tol=1e-04, verbose= verbose_level)
print("K-Means Clustering: Build", flush=True)
ykm = kmeans.fit(X)

Sat Nov 10 17:05:18 2018
K-Means Clustering: Start
K-Means Clustering: Build
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 72713.41045991046
start iteration
done sorting
end inner loop
Iteration 1, inertia 72139.95637254903
start iteration
done sorting
end inner loop
Iteration 2, inertia 71630.02661064426
start iteration
done sorting
end inner loop
Iteration 3, inertia 70222.6137254902
start iteration
done sorting
end inner loop
Iteration 4, inertia 67047.20915032679
start iteration
done sorting
end inner loop
Iteration 5, inertia 43635.388819357526
start iteration
done sorting
end inner loop
Iteration 6, inertia 43587.023411371236
start iteration
done sorting
end inner loop
Iteration 7, inertia 43587.023411371236
center shift 0.000000e+00 within tolerance 5.220518e-03
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 77739.90395256918
start iteration
done sorting
end inner loop
Iteration 1, inertia 74608

In [305]:
print(ykm.cluster_centers_, flush=True)

[[2.00756522e+03 7.23913043e+00 3.60869565e+00 1.38478261e+01
  2.06521739e+00 2.43478261e+01 1.65217391e+00 1.56521739e+00
  1.58695652e+00 1.45652174e+00 1.06304348e+01 1.65217391e+00
  2.30434783e+00 2.00328261e+03 1.17391304e+00 5.65217391e-01
  5.36086957e+01 1.16956522e+01 2.63043478e+00 1.36956522e+00]
 [2.00598077e+03 6.40384615e+00 4.07692308e+00 1.48269231e+01
  1.96153846e+00 2.21153846e+01 1.67307692e+00 1.76923077e+00
  1.59615385e+00 1.76923077e+00 1.04230769e+01 1.55769231e+00
  2.00000000e+00 1.99882692e+03 1.59615385e+00 5.19230769e-01
  2.15000000e+01 1.37307692e+01 2.48076923e+00 1.42307692e+00]
 [2.00350000e+03 5.00000000e+00 4.00000000e+00 1.55000000e+01
  1.00000000e+00 2.00000000e+00 1.00000000e+00 1.50000000e+00
  1.50000000e+00 2.00000000e+00 1.80000000e+01 9.90000000e+01
  4.50000000e+00 2.01700000e+03 1.00000000e+00 0.00000000e+00
  4.45000000e+01 9.90000000e+01 2.00000000e+00 3.00000000e+00]]


In [306]:
if pyscript:
    print(ykm.labels_, flush=True)
else:
    display(ykm.labels_, flush=True)

[1 1 1 1 1 1 0 0 0 0 0 1 1 1 0 0 1 1 1 1 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 1
 0 1 1 1 0 0 1 0 1 0 1 1 0 0 0 1 1 0 1 1 1 0 1 0 0 0 1 0 0 2 1 0 1 1 1 0 0
 0 0 0 1 0 1 0 0 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 2 1 1]


In [307]:
print("K-Means Clustering: End", flush=True)
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ), flush=True)

K-Means Clustering: End
Sat Nov 10 17:05:18 2018


## Feature selection using Random Forest

### Feature Selection 

In [308]:
print("Random Forest Feature Selection: Start", flush=True)
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ), flush=True)

from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=250, random_state=0, n_jobs=-1, verbose=verbose_level)
print("Random Forest Feature Selection: Fit Start", flush=True)
forest.fit(X, Y)
print("Random Forest Feature Selection: Fit", flush=True)

Random Forest Feature Selection: Start
Sat Nov 10 17:05:18 2018
Random Forest Feature Selection: Fit Start
building tree 1 of 1000
building tree 2 of 1000building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000

building tree 6 of 1000
building tree 7 of 1000building tree 8 of 1000

building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000building tree 14 of 1000
building tree 15 of 1000

building tree 16 of 1000
building tree 17 of 1000building tree 18 of 1000

building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000building tree 22 of 1000building tree 23 of 1000


building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000building tree 29 of 1000

building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000building tree 35 of 1000building tree 36 of 1000build

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:    0.0s


building tree 395 of 1000
building tree 396 of 1000
building tree 397 of 1000
building tree 398 of 1000
building tree 399 of 1000building tree 400 of 1000

building tree 401 of 1000building tree 402 of 1000

building tree 403 of 1000
building tree 404 of 1000
building tree 405 of 1000
building tree 406 of 1000
building tree 407 of 1000
building tree 408 of 1000
building tree 409 of 1000
building tree 410 of 1000
building tree 411 of 1000
building tree 412 of 1000
building tree 413 of 1000
building tree 414 of 1000
building tree 415 of 1000
building tree 416 of 1000
building tree 417 of 1000
building tree 418 of 1000
building tree 419 of 1000building tree 420 of 1000

building tree 421 of 1000building tree 422 of 1000
building tree 423 of 1000
building tree 424 of 1000

building tree 425 of 1000
building tree 426 of 1000
building tree 427 of 1000
building tree 428 of 1000
building tree 429 of 1000
building tree 430 of 1000
building tree 431 of 1000
building tree 432 of 1000building tree

[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    0.2s


building tree 796 of 1000
building tree 797 of 1000
building tree 798 of 1000building tree 799 of 1000building tree 800 of 1000


building tree 801 of 1000
building tree 802 of 1000
building tree 803 of 1000building tree 804 of 1000

building tree 805 of 1000
building tree 806 of 1000
building tree 807 of 1000
building tree 808 of 1000
building tree 809 of 1000building tree 810 of 1000
building tree 811 of 1000
building tree 812 of 1000

building tree 813 of 1000
building tree 814 of 1000
building tree 815 of 1000
building tree 816 of 1000
building tree 817 of 1000
building tree 818 of 1000
building tree 819 of 1000
building tree 820 of 1000
building tree 821 of 1000
building tree 822 of 1000
building tree 823 of 1000
building tree 824 of 1000
building tree 825 of 1000building tree 826 of 1000

building tree 827 of 1000
building tree 828 of 1000
building tree 829 of 1000
building tree 830 of 1000
building tree 831 of 1000
building tree 832 of 1000
building tree 833 of 1000
building tre

[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    0.4s finished


### Get the inportant features from random forest

In [309]:
importFeatures = forest.feature_importances_

### List the features by importancce

In [310]:
print("Random Forest Feature Selection: Feature Importance", flush=True)
print(importFeatures, flush=True)

Random Forest Feature Selection: Feature Importance
[0.09793474 0.08280231 0.07964555 0.08954928 0.03996557 0.0778428
 0.02267121 0.03549195 0.02617929 0.05023949 0.02655274 0.02561112
 0.0261996  0.10391637 0.02287142 0.02473135 0.11309343 0.02282479
 0.01383272 0.01804427]


In [311]:
indices = np.argsort(importFeatures)[::-1]
print(indices, flush=True)
featureLabel = X.columns[0:]
print(featureLabel, flush=True)
rankedFeature = []
for f in range(X.shape[1]):
    rankedFeature.append(featureLabel[indices[f]])
    print("%2d) %-*s %f" % (f+1, 30,  featureLabel[indices[f]], importFeatures[indices[f]]))
print(rankedFeature, flush=True)

[16 13  0  3  1  2  5  9  4  7 10 12  8 11 15 14 17  6 19 18]
Index(['C_YEAR', 'C_MNTH', 'C_WDAY', 'C_HOUR', 'C_VEHS', 'C_CONF', 'C_RCFG',
       'C_WTHR', 'C_RSUR', 'C_RALN', 'C_TRAF', 'V_ID', 'V_TYPE', 'V_YEAR',
       'P_ID', 'P_SEX', 'P_AGE', 'P_PSN', 'P_SAFE', 'P_USER'],
      dtype='object')
 1) P_AGE                          0.113093
 2) V_YEAR                         0.103916
 3) C_YEAR                         0.097935
 4) C_HOUR                         0.089549
 5) C_MNTH                         0.082802
 6) C_WDAY                         0.079646
 7) C_CONF                         0.077843
 8) C_RALN                         0.050239
 9) C_VEHS                         0.039966
10) C_WTHR                         0.035492
11) C_TRAF                         0.026553
12) V_TYPE                         0.026200
13) C_RSUR                         0.026179
14) V_ID                           0.025611
15) P_SEX                          0.024731
16) P_ID                           0.0228

### Reduce the number of features

In [312]:
#select features that contribute more than 0.05
#[df_cat.columns[0:df_cat.columns.size -1]]
X_Selected = X[rankedFeature[0:10]]

if pyscript:
    print(X_Selected, flush=True)
else:
    display(X_Selected)

t_end =  time.time()
print(time.asctime( time.localtime(t_end) ), flush=True)

print("Random Forest Feature Selection: End", flush=True)


        P_AGE V_YEAR C_YEAR C_HOUR C_MNTH C_WDAY C_CONF C_RALN C_VEHS C_WTHR
3564056    25   1993   2008      8     10      5     21      2      2      1
1569277     6   2002   2003      8      1      5     21      1      2      5
3834571    28   1989   2009     19      9      3      6      4      1      2
824407     16   1985   2001     23      2      5      6      1      1      1
3515329    12   2006   2008     20      8      7     35      1      2      2
3441590    31   1998   2008     13      6      2     35      1      6      1
5243339    59   2018   2014     13      6      7      4      4      1      3
4698727    55   2000   2012      7      8      2     36      1      2      1
484740     64   1991   2000     10      4      4     21      1      2      1
2112369    67   1993   2004     11      7      6      3      4      1      1
4270477    68   2018   2011      9      2      4     36      1      2      1
...       ...    ...    ...    ...    ...    ...    ...    ...    ...    ...

In [313]:
print("Split Test and Train based on Selected Features", flush=True)
#sprint into train and test 70/30
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_Selected, Y, test_size=0.3, random_state=0)

Split Test and Train based on Selected Features


### SVM GridSearch for Optimal Parms

In [314]:
#This operation is computationaly expensive.
#Enable as required.
enable_grid_search = False
if enable_grid_search:
    
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ), flush=True)
    
    from sklearn.grid_search import GridSearchCV
    from sklearn.svm import SVC
    param_grid = {'C':[0.1, 1, 10, 100, 1000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001]}
    grid = GridSearchCV(SVC(), param_grid, verbose=verbose_level)
    grid.fit(X_train, Y_train)
    print(grid.best_params_, flush=True)
    #{'C': 1000, 'gamma': 0.001}
    print(grid.best_estimator_, flush=True)
    #SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
    #  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    #  max_iter=-1, probability=False, random_state=None, shrinking=True,
    #  tol=0.001, verbose=False)
    grid_predictions = grid.predict(X_test)
    from sklearn.metrics import confusion_matrix
    confusion_matrix = confusion_matrix(Y_test, grid_predictions)
    print(confusion_matrix, flush=True)
    print(classification_report(Y_test,grid_predictions), flush=True)
    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ), flush=True)

## Logistic Regression Model

In [315]:
print("Logistic Regression: Start", flush=True)
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ), flush=True)
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1, random_state=0, verbose=verbose_level)
print("Logistic Regression: Fit", flush=True)
lr.fit(X_train, Y_train)

Logistic Regression: Start
Sat Nov 10 17:05:19 2018
Logistic Regression: Fit
[LibLinear]

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=3, warm_start=False)

In [316]:
print("Logistic Regression: Predict", flush=True)
y_pred = lr.predict(X_test)
#display(y_pred)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)), flush=True)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)), flush=True)

Logistic Regression: Predict
Accuracy of logistic regression classifier on train set: 0.61
Accuracy of logistic regression classifier on test set: 0.60


In [317]:
# print the intercept (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
print("Logistic Regression: Intercept", flush=True)
print(lr.intercept_, flush=True)

Logistic Regression: Intercept
[0.00423198]


In [318]:
# print the coeficients (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
print("Logistic Regression: Coefficients", flush=True)
print(lr.coef_, flush=True)

Logistic Regression: Coefficients
[[-0.01380873 -0.05702869  0.05901633  0.02350567 -0.09264553 -0.27877796
  -0.03890418  0.15946807 -0.61272426 -0.12121857]]


In [319]:
from sklearn.metrics import confusion_matrix
print("Logistic Regression: Confusion Matrix", flush=True)
confusion_matrix = confusion_matrix(Y_test, y_pred)
print(confusion_matrix, flush=True)
print("Logistic Regression: Classification Report", flush=True)
print(classification_report(Y_test,y_pred), flush=True)
t_end =  time.time()
print(time.asctime( time.localtime(t_end) ), flush=True)
print("Logistic Regression: End", flush=True)

Logistic Regression: Confusion Matrix
[[ 6 10]
 [ 2 12]]
Logistic Regression: Classification Report
             precision    recall  f1-score   support

          0       0.75      0.38      0.50        16
          1       0.55      0.86      0.67        14

avg / total       0.65      0.60      0.58        30

Sat Nov 10 17:05:19 2018
Logistic Regression: End


In [320]:
# with L1 regularization
print("Logistic Regression with L1 Regularization: Start", flush=True)
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ), flush=True)
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l1', C=1000, random_state=0, verbose=verbose_level)
print("Logistic Regression with L1 Regularization: Fit", flush=True)
lr.fit(X_train, Y_train)

Logistic Regression with L1 Regularization: Start
Sat Nov 10 17:05:19 2018
Logistic Regression with L1 Regularization: Fit
[LibLinear]

C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\svm\base.py:898: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=0, solver='liblinear', tol=0.0001,
          verbose=3, warm_start=False)

In [321]:
print("Logistic Regression with L1 Regularization: Predict", flush=True)
y_pred = lr.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)), flush=True)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)), flush=True)
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print("Logistic Regression with L1 Regularization: Confusion Matrix", flush=True)
print(confusion_matrix, flush=True)
print(classification_report(Y_test,y_pred), flush=True)
print("Logistic Regression with L1 Regularization: Classification Report", flush=True)
t_end =  time.time()
print(time.asctime( time.localtime(t_end) ), flush=True)
print("Logistic Regression with L1 Regularization: End", flush=True)

Logistic Regression with L1 Regularization: Predict
Accuracy of logistic regression classifier on train set: 0.64
Accuracy of logistic regression classifier on test set: 0.67
Logistic Regression with L1 Regularization: Confusion Matrix
[[ 7  9]
 [ 1 13]]
             precision    recall  f1-score   support

          0       0.88      0.44      0.58        16
          1       0.59      0.93      0.72        14

avg / total       0.74      0.67      0.65        30

Logistic Regression with L1 Regularization: Classification Report
Sat Nov 10 17:05:19 2018
Logistic Regression with L1 Regularization: End


### Decision Tree

In [322]:
print("Decision Tree: Start", flush=True)
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ), flush=True)
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion='entropy',max_depth=100, random_state=0)
print("Decision Tree: Fit", flush=True)
tree.fit(X_train, Y_train)

Decision Tree: Start
Sat Nov 10 17:05:19 2018
Decision Tree: Fit


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=100,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [323]:
print("Decision Tree: Predict", flush=True)
y_pred = tree.predict(X_test)
print('Accuracy of Decision Tree classifier on train set: {:.2f}'.format(tree.score(X_train, Y_train)), flush=True)
print('Accuracy of Decision Tree classifier on test set: {:.2f}'.format(tree.score(X_test, Y_test)), flush=True)

Decision Tree: Predict
Accuracy of Decision Tree classifier on train set: 1.00
Accuracy of Decision Tree classifier on test set: 0.60


In [324]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print("Decision Tree: Confusion Matrix", flush=True)
print(confusion_matrix, flush=True)
print("Decision Tree: Classification Report", flush=True)
print(classification_report(Y_test,y_pred), flush=True)
t_end =  time.time()
print(time.asctime( time.localtime(t_end) ), flush=True)
print("Decision Tree: End", flush=True)

Decision Tree: Confusion Matrix
[[ 8  8]
 [ 4 10]]
Decision Tree: Classification Report
             precision    recall  f1-score   support

          0       0.67      0.50      0.57        16
          1       0.56      0.71      0.63        14

avg / total       0.61      0.60      0.60        30

Sat Nov 10 17:05:19 2018
Decision Tree: End


### Random forest

In [325]:
print("Ensemble (Bagging): Random Forest: Start", flush=True)
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ), flush=True)
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(criterion='entropy', n_estimators=250, random_state=0, n_jobs=2, verbose=verbose_level)
print("Ensemble (Bagging): Random Forest: Fit", flush=True)
forest.fit(X_train, Y_train)

Ensemble (Bagging): Random Forest: Start
Sat Nov 10 17:05:19 2018
Ensemble (Bagging): Random Forest: Fit
building tree 1 of 1000
building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000building tree 5 of 1000

building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000building tree 27 of 1000

building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000building tree 33 of 1000

building tree 34 of 1000building tree 35 of 1000

building tree 36 of 1000
buil

[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 284 tasks      | elapsed:    0.0s


building tree 410 of 1000
building tree 411 of 1000
building tree 412 of 1000
building tree 413 of 1000
building tree 414 of 1000
building tree 415 of 1000
building tree 416 of 1000
building tree 417 of 1000
building tree 418 of 1000
building tree 419 of 1000
building tree 420 of 1000
building tree 421 of 1000
building tree 422 of 1000
building tree 423 of 1000
building tree 424 of 1000
building tree 425 of 1000
building tree 426 of 1000
building tree 427 of 1000
building tree 428 of 1000
building tree 429 of 1000
building tree 430 of 1000
building tree 431 of 1000
building tree 432 of 1000
building tree 433 of 1000
building tree 434 of 1000
building tree 435 of 1000
building tree 436 of 1000
building tree 437 of 1000
building tree 438 of 1000
building tree 439 of 1000
building tree 440 of 1000
building tree 441 of 1000
building tree 442 of 1000
building tree 443 of 1000
building tree 444 of 1000building tree 445 of 1000

building tree 446 of 1000
building tree 447 of 1000
building tre

[Parallel(n_jobs=2)]: Done 508 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s


building tree 805 of 1000
building tree 806 of 1000
building tree 807 of 1000
building tree 808 of 1000
building tree 809 of 1000
building tree 810 of 1000
building tree 811 of 1000
building tree 812 of 1000
building tree 813 of 1000
building tree 814 of 1000
building tree 815 of 1000
building tree 816 of 1000
building tree 817 of 1000
building tree 818 of 1000
building tree 819 of 1000
building tree 820 of 1000
building tree 821 of 1000
building tree 822 of 1000
building tree 823 of 1000
building tree 824 of 1000
building tree 825 of 1000building tree 826 of 1000

building tree 827 of 1000
building tree 828 of 1000
building tree 829 of 1000building tree 830 of 1000

building tree 831 of 1000
building tree 832 of 1000
building tree 833 of 1000
building tree 834 of 1000
building tree 835 of 1000building tree 836 of 1000

building tree 837 of 1000building tree 838 of 1000

building tree 839 of 1000building tree 840 of 1000

building tree 841 of 1000building tree 842 of 1000

building tre

[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.4s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=2,
            oob_score=False, random_state=0, verbose=3, warm_start=False)

In [326]:
print("Ensemble (Bagging): Random Forest: Predict", flush=True)
y_pred = forest.predict(X_test)
print('Accuracy of RandomForest classifier on train set: {:.2f}'.format(forest.score(X_train, Y_train)), flush=True)
print('Accuracy of RandomForest classifier on test set: {:.2f}'.format(forest.score(X_test, Y_test)), flush=True)

Ensemble (Bagging): Random Forest: Predict


[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 284 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 508 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 284 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 508 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 284 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 508 tasks      | elapsed:    0.0s
[P

Accuracy of RandomForest classifier on train set: 1.00
Accuracy of RandomForest classifier on test set: 0.70


In [327]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print("Ensemble (Bagging): Random Forest: Confusion Matrix", flush=True)
print(confusion_matrix, flush=True)
print("Ensemble (Bagging): Random Forest: Classification Report", flush=True)
print(classification_report(Y_test,y_pred), flush=True)
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ), flush=True)
print("Ensemble (Bagging): Random Forest: End", flush=True)

Ensemble (Bagging): Random Forest: Confusion Matrix
[[ 9  7]
 [ 2 12]]
Ensemble (Bagging): Random Forest: Classification Report
             precision    recall  f1-score   support

          0       0.82      0.56      0.67        16
          1       0.63      0.86      0.73        14

avg / total       0.73      0.70      0.69        30

Sat Nov 10 17:05:21 2018
Ensemble (Bagging): Random Forest: End


### K-N-N

In [328]:
print("KNN: Start", flush=True)
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ), flush=True)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
print("KNN: Fit", flush=True)
knn.fit(X_train, Y_train)

KNN: Start
Sat Nov 10 17:05:21 2018
KNN: Fit


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [329]:
print("KNN: Predict", flush=True)
y_pred = knn.predict(X_test)
print('Accuracy of KNN classifier on train set: {:.2f}'.format(knn.score(X_train, Y_train)), flush=True)
print('Accuracy of KNN classifier on test set: {:.2f}'.format(knn.score(X_test, Y_test)), flush=True)

KNN: Predict
Accuracy of KNN classifier on train set: 0.66
Accuracy of KNN classifier on test set: 0.53


In [330]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print("KNN: Confusion Matrix", flush=True)
print(confusion_matrix, flush=True)
print("KNN: Classification Report", flush=True)
print(classification_report(Y_test,y_pred), flush=True)
t_end =  time.time()
print(time.asctime( time.localtime(t_end) ), flush=True)
print("KNN: End", flush=True)

KNN: Confusion Matrix
[[8 8]
 [6 8]]
KNN: Classification Report
             precision    recall  f1-score   support

          0       0.57      0.50      0.53        16
          1       0.50      0.57      0.53        14

avg / total       0.54      0.53      0.53        30

Sat Nov 10 17:05:21 2018
KNN: End


### SVM

In [331]:
print("SVM: Start", flush=True)
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ), flush=True)
from sklearn.svm import SVC
#svm = SVC(C=1, random_state=0, kernel='sigmoid', verbose=True)
#svm = SVC(C=1, random_state=0, kernel='linear', verbose=True, cache_size=200)
svm = SVC(verbose = verbose_level)
print("SVM: Fit", flush=True)
svm.fit(X_train, Y_train)

SVM: Start
Sat Nov 10 17:05:21 2018
SVM: Fit
[LibSVM]

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=3)

In [332]:
print("SVM: Predict", flush=True)
y_pred = svm.predict(X_test)
print('Accuracy of SVM classifier on train set: {:.2f}'.format(svm.score(X_train, Y_train)), flush=True)
print('Accuracy of SVM classifier on test set: {:.2f}'.format(svm.score(X_test, Y_test)), flush=True)

SVM: Predict
Accuracy of SVM classifier on train set: 1.00
Accuracy of SVM classifier on test set: 0.47


In [333]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print("SVM: Confusion Matrix", flush=True)
print(confusion_matrix, flush=True)
print("SVM: Classfication Report", flush=True)
print(classification_report(Y_test,y_pred), flush=True)
t_end =  time.time()
print(time.asctime( time.localtime(t_end) ), flush=True)
print("SVM: End", flush=True)

SVM: Confusion Matrix
[[ 0 16]
 [ 0 14]]
SVM: Classfication Report
             precision    recall  f1-score   support

          0       0.00      0.00      0.00        16
          1       0.47      1.00      0.64        14

avg / total       0.22      0.47      0.30        30

Sat Nov 10 17:05:21 2018
SVM: End


C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Performance Tuning using GridSearch

In [334]:
enable_grid_search = True
if enable_grid_search:
    print("Grid Search (SVM): Start", flush=True)
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ), flush=True)
    from sklearn.grid_search import GridSearchCV
    from sklearn.svm import SVC
    param_grid = {'C':[0.1, 1, 10, 100, 1000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001]}
    grid = GridSearchCV(SVC(), param_grid, verbose=verbose_level)
    print("Grid Search (SVM): Fit", flush=True)
    grid.fit(X_train, Y_train)
    print(grid.best_params_, flush=True)
    #Result: {'C': 1000, 'gamma': 0.001}
    print(grid.best_estimator_, flush=True)
        #SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
        #decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
        #max_iter=-1, probability=False, random_state=None, shrinking=True,
        #tol=0.001, verbose=False)
    print("Grid Search (SVM): Predict", flush=True)
    grid_predictions = grid.predict(X_test)
    from sklearn.metrics import confusion_matrix
    confusion_matrix = confusion_matrix(Y_test, grid_predictions)
    print("Grid Search (SVM): Confusion Matrix", flush=True)
    print(confusion_matrix, flush=True)
    print("Grid Search (SVM): Classification Report", flush=True)
    print(classification_report(Y_test,grid_predictions), flush=True)
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ), flush=True)
    print("Grid Search (SVM): End", flush=True)
    
    
#[[4810 1501  697]
# [3549 2112 1362]
# [ 801  905 5036]]
#             precision    recall  f1-score   support
#
#          1       0.53      0.69      0.60      7008
#          2       0.47      0.30      0.37      7023
#          3       0.71      0.75      0.73      6742
#
#avg / total       0.57      0.58      0.56     20773

Grid Search (SVM): Start
Sat Nov 10 17:05:21 2018
Grid Search (SVM): Fit
Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] C=0.1, gamma=1 ..................................................
[CV] ......................... C=0.1, gamma=1, score=0.583333 -   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ......................... C=0.1, gamma=1, score=0.565217 -   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ......................... C=0.1, gamma=1, score=0.565217 -   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ....................... C=0.1, gamma=0.1, score=0.583333 -   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ....................... C=0.1, gamma=0.1, score=0.565217 -   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ....................... C=0.1, gamma=0.1, score=0.565217 -   0.0s
[CV] C=0.1, ga

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ...................... C=1, gamma=0.0001, score=0.583333 -   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ...................... C=1, gamma=0.0001, score=0.565217 -   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ...................... C=1, gamma=0.0001, score=0.565217 -   0.0s
[CV] C=10, gamma=1 ...................................................
[CV] .......................... C=10, gamma=1, score=0.583333 -   0.0s
[CV] C=10, gamma=1 ...................................................
[CV] .......................... C=10, gamma=1, score=0.565217 -   0.0s
[CV] C=10, gamma=1 ...................................................
[CV] .......................... C=10, gamma=1, score=0.565217 -   0.0s
[CV] C=10, gamma=0.1 .................................................
[CV] ........................ C=10, gamma=0.1, score=0.583333 -   0.0s
[CV] C=10, gamma=0.1 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.3s finished


## ANN - Multilayer Perceptron

In [335]:
print("Multilayer Preceptron: Start", flush=True)
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ), flush=True)
from sklearn.neural_network import MLPClassifier
#mlpc = MLPClassifier(alpha=1)
mlpc = MLPClassifier(hidden_layer_sizes=(12, 12, 12), max_iter=1000, verbose=verbose_level)

#mlp = multilayer_perceptron(n_hidden =2, activation='logistic', algorithm='sgd', random_state=3)
print("Multilayer Preceptron: fit", flush=True)
mlpc.fit(X_train, Y_train)

Multilayer Preceptron: Start
Sat Nov 10 17:05:21 2018
Multilayer Preceptron: fit
Iteration 1, loss = 1.99272834
Iteration 2, loss = 2.66608651
Iteration 3, loss = 2.77457641
Iteration 4, loss = 1.61476438
Iteration 5, loss = 1.22109955
Iteration 6, loss = 2.03433066
Iteration 7, loss = 1.68823778
Iteration 8, loss = 1.04318767
Iteration 9, loss = 1.32812940
Iteration 10, loss = 1.63913845
Iteration 11, loss = 1.49174306
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(12, 12, 12), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=3, warm_start=False)

In [336]:
print("Multilayer Preceptron: Predict", flush=True)
y_pred = mlpc.predict(X_test)
print('Accuracy of Multilayer Perceptron classifier on train set: {:.2f}'.format(mlpc.score(X_train, Y_train)), flush=True)
print('Accuracy of Multilayer Perceptron classifier on test set: {:.2f}'.format(mlpc.score(X_test, Y_test)), flush=True)

Multilayer Preceptron: Predict
Accuracy of Multilayer Perceptron classifier on train set: 0.46
Accuracy of Multilayer Perceptron classifier on test set: 0.43


In [337]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print("Multilayer Preceptron: Confusion Matrix", flush=True)
print(confusion_matrix, flush=True)
print("Multilayer Preceptron: Classificiation Report", flush=True)
print(classification_report(Y_test,y_pred), flush=True)
t_end =  time.time()
print(time.asctime( time.localtime(t_end) ), flush=True)
print("Multilayer Preceptron: End", flush=True)

Multilayer Preceptron: Confusion Matrix
[[ 2 14]
 [ 3 11]]
Multilayer Preceptron: Classificiation Report
             precision    recall  f1-score   support

          0       0.40      0.12      0.19        16
          1       0.44      0.79      0.56        14

avg / total       0.42      0.43      0.36        30

Sat Nov 10 17:05:21 2018
Multilayer Preceptron: End


In [338]:

#check sigmoid and rbf
#from sklearn.ensemble import BaggingClassifier
#from sklearn.svm import SVC
#clf = BaggingClassifier(SVC(C=1.0,
#        cache_size=200,
#        class_weight=None,
#        coef0=0.0,
#        decision_function_shape=None,
#        degree=3,
#        gamma='auto',
#        kernel='linear',
#        max_iter=-1,
#        probability=False,
#        random_state=None,
#        shrinking=True,
#        tol=0.001,
#        verbose=False,
#        ))

In [339]:
#clf.fit(X_train, Y_train)

In [340]:
t_end =  time.time()
print(time.asctime( time.localtime(t_end) ), flush=True)

Sat Nov 10 17:05:21 2018


In [ ]:
sys.stdout.close()